# Traditional Chinese Medicine Tools Demo

This notebook demonstrates how to leverage TCM tools for analyzing and querying traditional Chinese medicine information from multiple databases. We'll explore:

- Querying herb and formula information from TCM databases
- Creating customized herbal formulas
- Analyzing herb properties and interactions
- Generating patient-specific recommendations

## Prerequisites
- OpenAI API key in `.env` file
- `just-agents` library installed
- Python 3.11 environment

In [1]:
# Load environment variables and core dependencies
from dotenv import load_dotenv
load_dotenv()  # This loads OpenAI API key from .env

# Import required libraries
from just_agents.base_agent import BaseAgent
import json
from agents.tcm_crew import TCMTools, find_TCM_herb, find_TCM_condition

## Initialize TCM Tools

`TCMTools` will load necessary data from HuggingFace. This includes:
- BATMAN-TCM database
- DragonTCM dataset
- Cross-reference mappings

First initialization may take several minutes to download and cache the data.
After initialization, `TCMTools` will be accessible to the all tools your TCM agent has access to. 

In [2]:
# Initialize TCM toolset with database connections
tcm_tools = TCMTools()

# Download and initialize databases
tcm_tools.initialize()

/home/f.galkin/miniconda3/envs/agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


📂 Accessing HuggingFace repository...
✅ Connector for BATMAN-TCM2 and DragonTCM has been loaded
Start processing individual databases:
Adding cross mappings
Done adding cross mappings
Adding equivalents
Done adding equivalents
Loading DB: BATMAN
⚠️ Loading this dataset usually takes ~3-5 minutes.
⏳ Downloading compressed database file from HuggingFace...
📂 Database file downloaded, now decompressing and loading...
✅ JSON file has been read!
Loading the DB into memory...
✅ Database has been loaded successfully!
Uploaded DB: BATMAN
Loading DB: American Dragon
Added 1044 herbs
Added 1119 conditions
Added 2580 formulas
Uploaded DB: American Dragon
Adding word maps
Done adding word maps
from_huggingface execution time: 50.81 seconds


## Formula Analysis Setup

We'll analyze a custom herbal formula and create personalized recommendations based on patient profile.
The formula components are stored in a JSON file that we've obtained in the [previous demo notebook](1.TCMxPlore%20Agent%20Annotation.ipynb).

To make the final agent output more relevant, we'll use two agents:
1. An agent that frames a patien't health situation in the context of TCM;
2. An agent that uses this information and a given formula to modify it for this particular patient;
3. An agent that distills all the specifics of the case into one prescription.

In [3]:
# Load formula components from JSON file
with open("./31Oct2024_floral_fla_annots.json") as f:
    herbal_fla = json.load(f)
    
# Extract herb names from the formula data
components = [x['init']['preferred_name'] for x in herbal_fla]

# Define a sample patient profile for personalized recommendations
person = '''30 year old white male with sedentary lifestyle,
           BMI=21, suffering from IBD and headaches'''

# Define the task for the agent that will provide TCM context for any conditions
# present in the patient
physician_task = f'''
You are consulting a patient with the following profile {person}.
Retrieve information about their medical conditions from a database of diseases explained in TCM terms 
and format it as a reference for a TCM practitioner who will use it to design a herbal formula for the 
patient. You may try looking up for different wording used to describe the conditions in the patient's 
anamnesis (e.g. full names instead of abbreviations)

In your report present:
- herbs that fit the patient's profile the best (<5 herbs);
- general TCM principles that need to be counteracted in the patient;
- herbs that need to be avoided by this person.


'''

# Create detailed the task description for formula modification
tcm_task = f"""
You are a TCM practitioner analyzing a formula with herbs: 
{components}. 
Consult with databases to gain more understanding of the components of this formula.

This formula will be given to {person}

In your description of a recipe, provide a breakdown of:
* Dosages of all herbs to be included in a formula (weight and form of component, skip detailed explanations);
* Formula consumption regimen (before or after food, times per day, maximal dose, 
recommended dose, course duration, time of the day to take it)
* Indications and conditions that this formula may help with;
* Actions of this formula explained in TCM forms;
* Contraindications for this formula
* How it may affect the basic aging processes in a human body
* Modifications: Suggest 1-2 other herbs for the patient to consider to address their particular needs.

Consider the patient's specific needs, how various herbs can affect any preexisting conditions,
and suggest modfications to the original formula that are based on the following information from 
their personal physician: 
%s

"""

pharmacy_task = f"""
Given the detailed overview of TCM recommendations given to the patient ({person}), present an 
"optimal formula" the patient can take to a TCM pharmacy, which considers all the facts 
and specifics of their case.

Here are the details of their case:
%s

The optimal formula needs to be brief and contain exact amounts and forms of all ingredients for a 
pharmacist to prepare the final product. Provide minimalistic output with minimal commentary on decisions 
made.
"""

## Configure AI Agent

We'll use GPT-4 Turbo with our TCM tools to analyze the formula, relying on the convenient syntaxis defined in the [just-agents](https://github.com/longevity-genie/just-agents/tree/main) library.

The agents combine:
- Language model capabilities
- Structured database access
- Clinical reasoning tools

In [4]:
# Configure GPT-4 Turbo settings
GPT4TURBO = {
    "model": "gpt-4-turbo-preview",
    "temperature": 0.4,  # Lower temperature for more focused outputs
    "tools": []
}

personal_physician = BaseAgent(
                    llm_options=GPT4TURBO,
                    tools=[find_TCM_condition], # Define the functions this agent may call
                    completion_max_tries=3 # Retry failed completions
                )

tcm_doctor = BaseAgent(
                    llm_options=GPT4TURBO,
                    tools=[find_TCM_herb], 
                    completion_max_tries=3  
                )

tcm_pharmacist = BaseAgent(
                    llm_options=GPT4TURBO,
                    tools=[find_TCM_herb, find_TCM_condition], 
                    completion_max_tries=3  
                )

## Generate Formula Analysis

Now we'll generate a comprehensive analysis of our formula using the configured agent.
The analysis will include dosage recommendations, safety considerations, and patient-specific modifications.

The agents will receive information sequentially to help each other provide more relevant recommendations.

In [5]:
%%time
# 1. Ask the physician to provide context for the patient's health
exam_result = personal_physician.query(physician_task)
# 2. Ask the TCM specialist to adjust the formula for the patient
tcm_result = tcm_doctor.query(tcm_task%exam_result)
# 3. Ask the pharmacist to distill the established context into a brief TCM formula
prescription = tcm_pharmacist.query(pharmacy_task%tcm_result)

CPU times: user 79.3 ms, sys: 215 μs, total: 79.5 ms
Wall time: 57.4 s


You may now inspect the output of the two agents:

In [6]:
print(exam_result)

Based on the information retrieved from the TCM database regarding Inflammatory Bowel Disease (IBD) and headaches, here is a reference for a TCM practitioner to design a herbal formula for the patient. This 30-year-old white male with a sedentary lifestyle, a BMI of 21, suffering from IBD and headaches, will benefit from a careful selection of herbs, adherence to general TCM principles, and avoidance of certain herbs that may exacerbate his conditions.

### Herbs that Fit the Patient's Profile Best

1. **Bai Zhu (Atractylodes macrocephala)** - Supports the spleen and stomach, aids in digestion, and addresses dampness, which can be beneficial for IBD.
2. **Huang Qin (Scutellaria baicalensis)** - Has anti-inflammatory properties and can help with both IBD and headaches caused by heat.
3. **Chuan Xiong (Ligusticum wallichii)** - Moves Qi and Blood, alleviating pain, making it suitable for headaches.
4. **Ge Gen (Pueraria lobata)** - Releases the exterior and muscle layer, beneficial for h

In [7]:
print(tcm_result)

Based on the information retrieved from the TCM database, here is a detailed breakdown of the original formula consisting of Ren Shen, Lu Dou, and Ma Huang, tailored for a 30-year-old male with a sedentary lifestyle, BMI=21, suffering from IBD and headaches. Additionally, modifications are suggested to address the patient's specific needs.

### Original Formula Components

1. **Ren Shen (Ginseng Root)**
   - **Dosage**: Decoction: 1-9g; Powder: 0.5-1g; Tincture: 0.25-10ml
   - **Actions**: Tonifies Yuan Qi, Spleen, and Stomach Qi; generates body fluids; calms the spirit.
   - **Contraindications**: Yin deficiency with heat signs, hypertension, early stages of bleeding disorders.

2. **Lu Dou (Mung Bean)**
   - **Dosage**: Dry: 15-30g
   - **Actions**: Resolves summerheat, clears heat, relieves toxins.
   - **Contraindications**: Loose stools or diarrhea; not to be used unless there is heat.

3. **Ma Huang (Ephedra Stem)**
   - **Dosage**: Dry: 1.5-10g; Tincture: 1-3ml
   - **Actions**:

In [8]:
print(prescription)

**Optimal TCM Formula for IBD and Headaches**

1. **Ren Shen (Ginseng Root)**
   - **Dosage**: 5g (decoction)

2. **Lu Dou (Mung Bean)**
   - **Dosage**: 20g (dry)

3. **Ma Huang (Ephedra Stem)**
   - **Dosage**: Avoid due to potential exacerbation of hypertension and restlessness.

4. **Bai Zhu (Atractylodes macrocephala)**
   - **Dosage**: 10g (decoction)

5. **Huang Qin (Scutellaria baicalensis)**
   - **Dosage**: 10g (decoction)

**Instructions for Use**

- **Dosage**: Combine the specified amounts of each herb for a single dose.
- **Times Per Day**: Twice daily.
- **Course Duration**: 10-14 days, then reassess.
- **Time of Day**: Morning and evening.
- **Before or After Food**: After food.
- **Preparation**: Decoct Ren Shen, Bai Zhu, and Huang Qin in water for 30-40 minutes. Add Lu Dou towards the end of the boiling process.


## Examine Agent Process

For debugging and understanding, we can inspect how the agent arrived at its conclusions
by examining the conversation history and tool usage.

In [9]:
# Access agent memory to see tool usage
tool_calls = personal_physician.memory.__dict__['messages'][2]
# Display which tools were called and in what order
print("Tool execution sequence for the 'personal physician' agent:")
for call in tool_calls['tool_calls']:
    print(f"- {call['function']['name']}: {call['function']['arguments']}")

tool_calls = tcm_doctor.memory.__dict__['messages'][2]
print("\nTool execution sequence for the 'TCM practitioner' agent:")
for call in tool_calls['tool_calls']:
    print(f"- {call['function']['name']}: {call['function']['arguments']}")
    
# the pharmacist agent made no tool calls

Tool execution sequence for the 'personal physician' agent:
- find_TCM_condition: {"query": "Inflammatory Bowel Disease"}
- find_TCM_condition: {"query": "headaches"}

Tool execution sequence for the 'TCM practitioner' agent:
- find_TCM_herb: {"query": "REN SHEN"}
- find_TCM_herb: {"query": "LU DOU"}
- find_TCM_herb: {"query": "MA HUANG"}


Notice that the initial formula contained ephedra, which was removed due to the patient's complaints about headaches. This brief workflow shows how AI agents can provide extra personalization and adjust the formulas designed based on rigid scripts.